# Contents
[Order of the SELECT Statement Clauses](#Order-of-the-SELECT-Statement-Clauses) &emsp; [Clause Execution Order](#Clause-Execution-Order)

In [33]:
# Left-align all the tables
from IPython.core.display import HTML
table_css = 'table {align:left;display:block} '
HTML('<style>{}</style>'.format(table_css))

In [31]:
import saspy               # Import the saspy module
sas = saspy.SASsession()   # Connect to the SAS environment

Using SAS Config named: oda
SAS Connection established. Subprocess id is 20316



In [32]:
%%SAS
libname sql '~/sql';   *Assign a library reference;
title;                 *No title for all procedures output;

**PROC SQL** is a database language. The **SELECT statement** is often referred to as an **SQL query** and is used to read data from one ore more database tables (or data sets). It has only one clause that is required to be specified -- the **FROM clause**. 

## Order of the SELECT Statement Clauses
1. SQL = **SELECT**
2. Is  = **INTO**
3. Fun = **FROM**
4. When   = **WHERE**
5. Geeks  = **GROUP BY**
6. Help  = **HAVING**
7. Others = **ORDER BY**

## Clause Execution Order
<table>
  <tr>
    <th>Clause Execution Order </th>
    <th>Description</th>
  </tr>
  <tr>
    <td>1. FROM Clause</td>
    <td>The FROM clause is a required clause with the purpose of determining the working set of data that is being queried.</td>
  </tr>
  <tr>
    <td>2. INTO Clause</td>
    <td>The INTO clause is used to create one or more macro variables where the values can be used to manipulate data in DATA and PROC steps.</td>
  </tr>
    <tr>
    <td>3. WHERE Clause</td>
    <td>The WHERE clause is used to subset rows of data based on the condition(s) specified, and rows that aren't satisfied by the condition(s) are discarded.</td>
  </tr>
    <tr>
    <td>4. GROUP BY Clause</td>
    <td>The GROUP BY clause takes the rows that were subset with the WHERE clause and grouped based on common values in the column specified in the GROUP BY clause. </td>
  </tr>
    <tr>
    <td>5. HAVING Clause</td>
    <td>The HAVING clause applies the condition(s) to the grouped rows specified in the GROUP BY clause, and any grouped rows that aren't satisfied by the condition(s) are discarded.</td>
  </tr>
    <tr>
    <td>6. SELECT Statement</td>
    <td>Expressions specified in the SELECT statement are processed.</td>
  </tr>
    <tr>
    <td>7. ORDER BY Clause </td>
    <td>The ORDER BY clause sorts the rows of data in either ascending (default) or descending order.</td>
  </tr>
</table>

## SAS Data Types 
* **Numeric data** are assigned a default length of 8 bytes.
* **Character data** are assigned a default length of 8 characters. 

## Creating Column Aliases
In situations where data are computed using system functions, statistical functions, or arithmetic operations, a column name or header can be left blank. To prevent this from occurring, users may specify the **AS keyword** to provide a name to the column or heading itself along with formatting directions.

In [4]:
%%SAS
proc sql;
    select prodname,
           prodtype,
           prodcost * 0.80 as Discount_Price format=dollar9.2
        from sql.products
            order by 3;
quit;

Product Name,Product Type,Discount_Price
Analog Cell Phone,Phone,$28.00
Office Phone,Phone,$104.00
Digital Cell Phone,Phone,$140.00
Spreadsheet Software,Software,$239.20
Graphics Software,Software,$239.20
Wordprocessor Software,Software,$239.20
Database Software,Software,$319.20
Dream Machine,Workstation,"$2,560.00"
Business Machine,Workstation,"$2,640.00"
Travel Laptop,Laptop,"$2,720.00"


## Finding Duplicate Values
In some situations, several rows in a table may contain identical or duplicate column values. To select only one of each identical or duplicate values, SAS supports and processes the **DISTINCT** and **UNIQUE keywords** the same and without any noticeable performance differences. 

In [5]:
%%SAS
proc sql;
    select distinct /*unique*/ manunum
        from sql.inventory;
quit;

Manufacturer Number
111
170
500
600


## Comparison Operators
**Comparison operators** are used in the SQL procedure to compare one character or numeric value to another. 

<table>
  <tr>
    <th>SAS Operator</th>
    <th>Mnemonic Operators</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>=</td>
    <td>EQ</td>
    <td>Equal to</td>
  </tr>
  <tr>
    <td>^= or ~=</td>
    <td>NE</td>
    <td>Not equal to</td>
  </tr>
    <tr>
    <td><</td>
    <td>LT</td>
    <td>Less than</td>
  </tr><tr>
    <td><=</td>
    <td>LE</td>
    <td>Less than or equal to</td>
  </tr><tr>
    <td>></td>
    <td>GT</td>
    <td>Greater than</td>
  </tr><tr>
    <td>>=</td>
    <td>GE</td>
    <td>Greater than or equal to</td>
  </tr>
</table>

In [6]:
%%SAS
*Select only those products from the PRODUCTS table that cost more than $300.00;
proc sql;
    select prodname, 
           prodtype,
           prodcost
        from sql.products
            where prodcost > 300;
quit;

Product Name,Product Type,Product Cost
Dream Machine,Workstation,"$3,200.00"
Business Machine,Workstation,"$3,300.00"
Travel Laptop,Laptop,"$3,400.00"
Database Software,Software,$399.00


## Logical Operators
**Logical operators** are used to connect two or more expressions together in a WHERE or HAVING clause. The available logical operators consist of **AND**, **OR**, and **NOT**. 

In [7]:
%%SAS
*Select only those software products that cost more than $300.00;
proc sql;
    select prodname, 
           prodtype,
           prodcost
        from sql.products
            where prodtype = 'Software' and 
                  prodcost > 300;
quit;

Product Name,Product Type,Product Cost
Database Software,Software,$399.00


In [8]:
%%SAS
*Select software products or products that cost more than $300.00;
proc sql;
    select prodname, 
           prodtype, 
           prodcost
        from sql.products;
            where prodtype = 'Software' or 
                  prodcost > 300;
quit;

Product Name,Product Type,Product Cost
Dream Machine,Workstation,"$3,200.00"
Business Machine,Workstation,"$3,300.00"
Travel Laptop,Laptop,"$3,400.00"
Analog Cell Phone,Phone,$35.00
Digital Cell Phone,Phone,$175.00
Office Phone,Phone,$130.00
Spreadsheet Software,Software,$299.00
Database Software,Software,$399.00
Wordprocessor Software,Software,$299.00
Graphics Software,Software,$299.00


In [9]:
%%SAS
*Select products that are not software products and do not cost more than $300.00;
proc sql;
    select prodname,
           prodtype,
           prodcost
        from sql.products;
            where not prodtype = 'Software' and 
                  not prodcost > 300;
quit;

Product Name,Product Type,Product Cost
Dream Machine,Workstation,"$3,200.00"
Business Machine,Workstation,"$3,300.00"
Travel Laptop,Laptop,"$3,400.00"
Analog Cell Phone,Phone,$35.00
Digital Cell Phone,Phone,$175.00
Office Phone,Phone,$130.00
Spreadsheet Software,Software,$299.00
Database Software,Software,$399.00
Wordprocessor Software,Software,$299.00
Graphics Software,Software,$299.00


## Arithmetic Operators
<table>
  <tr>
    <th>Operator </th>
    <th>Description</th>
  </tr>
  <tr>
    <td>+</td>
    <td>Addition</td>
  </tr>
  <tr>
    <td>-</td>
    <td>Subtraction</td>
  </tr>
    <tr>
    <td>*</td>
    <td>Multiplication</td>
  </tr>
    <tr>
    <td>/</td>
    <td>Division</td>
  </tr>
    <tr>
    <td>**</td>
    <td>Exponentiation</td>
  </tr>
    <tr>
    <td>=</td>
    <td>Equality</td>
  </tr>
</table>

In [10]:
%%SAS
*Apply a discount of 20% to the product price (PRODCOST) in the PRODUCTS table;
proc sql;
    select prodname, 
           prodtype, 
           prodcost * 0.80 as Dicount_Price
        from sql.products;
quit;

Product Name,Product Type,Dicount_Price
Dream Machine,Workstation,2560
Business Machine,Workstation,2640
Travel Laptop,Laptop,2720
Analog Cell Phone,Phone,28
Digital Cell Phone,Phone,140
Office Phone,Phone,104
Spreadsheet Software,Software,239.2
Database Software,Software,319.2
Wordprocessor Software,Software,239.2
Graphics Software,Software,239.2


PROC SQL allows references to a computed column in the same SELECT statement (or a WHERE clause) using the **CALCULATED keyword**. 

In [11]:
%%SAS
*Apply a discount of 20% to the product price (PRODCOST) in the PRODUCTS table, 
 calculate the loss using the discounted price, and
 sort the resulting set by ascending order of discounted price;
proc sql;
    select prodname,
           prodtype,
           prodcost * 0.80 as Discount_Price
               format=dollar9.2,
           prodcost - calculated Discount_Price as Loss 
               format=dollar7.2
        from sql.products
            order by 3;
run;

Product Name,Product Type,Discount_Price,Loss
Analog Cell Phone,Phone,$28.00,$7.00
Office Phone,Phone,$104.00,$26.00
Digital Cell Phone,Phone,$140.00,$35.00
Spreadsheet Software,Software,$239.20,$59.80
Graphics Software,Software,$239.20,$59.80
Wordprocessor Software,Software,$239.20,$59.80
Database Software,Software,$319.20,$79.80
Dream Machine,Workstation,"$2,560.00",$640.00
Business Machine,Workstation,"$2,640.00",$660.00
Travel Laptop,Laptop,"$2,720.00",$680.00


## Summarizing Data
<table>
  <tr>
    <th>Summary Function</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>AVG, MEAN</td>
    <td>Average or mean of values</td>
  </tr>
  <tr>
    <td>COUNT, FREQ, N</td>
    <td>Aggreagate number of non-missing values</td>
  </tr>
  <tr>
    <td>CSS</td>
    <td>Corrected sum of squares</td>
  </tr>
  <tr> 
    <td>CV</td>
    <td>Coefficient of variation</td>
  </tr>
  <tr>
    <td>MAX</td>
    <td>Largest value</td>
  </tr>
  <tr>
    <td>MIN</td>
    <td>Smalles value</td>
  </tr>
  <tr>
    <td>NMISS</td>
    <td>Number of missing values</td>
  </tr>
  <tr>
    <td>PRT</td>
    <td>Probability of a greater absolute value of Student's t</td>
  </tr>
  <tr>
    <td>RANGE</td>
    <td>Difference between the largest and smallest values</td>
  </tr>
  <tr>
    <td>STD</td>
    <td>Standard deviation</td>
  </tr>
  <tr>
    <td>STDERR</td>
    <td>Standard error of the mean</td>
  </tr>
  <tr>
    <td>SUM</td>
    <td>Sum of values</td>
  </tr>
  <tr>
    <td>SUMWGT</td>
    <td>Sum of the weight values which is 1</td>
  </tr>
  <tr>
    <td>T</td>
    <td>Testing the hypothesis that the population mean is zero</td>
  </tr>
  <tr>
    <td>USS</td>
    <td>Uncorrected sum of squares</td>
  </tr>
  <tr>
    <td>VAR</td>
    <td>Variance</td>
  </tr>
</table>

In [12]:
%%SAS
*Count the total number of missing and nonmissing rows in the PURCHASES table;
proc sql;
    select count(*) as Row_Count
        from sql.purchases;
run;

*Count the total number of nonmissing rows;
proc sql;
    select count(units) as Non_Missing_Row_Count
        from sql.purchases;
run;

Row_Count
57
Non_Missing_Row_Count
57


In [13]:
%%SAS
*What is the least expensive product in the PRODUCTS table?;
proc sql;
    select min(prodcost) as Cheapest
            format=dollar9.2 Label='Least Expensive'
        from sql.products;
quit;

Least Expensive
$35.00


In [14]:
%%SAS
*Determine the total costs of all purchases by customers who bought 
workstation (PRODNUM=1110 and 1200) and laptops (PRODNUM=1700);
proc sql;
    select sum((units) * (unitcost))
            as Total_Purchases format=dollar12.2
        from sql.purchases;
            where prodnum = 1100 or 
                  prodnum = 1200 or 
                  prodnum = 1700;
quit;

Total_Purchases
"$295,473.00"


In [15]:
%%SAS
*Determine the average product for all software in the PRODUCTS table 
 containing a variety of products;
proc sql;
    select avg(prodcost) as 
            Average_Product_Cost format=dollar10.2
        from sql.products;
            where upcase(prodtype) in 
                    ('Software');
quit;

Average_Product_Cost
"$1,153.60"


In [16]:
%%SAS
*Determine the average cost of products in inventory;
proc sql;
    select prodnum,
           (invprice / invqty) as 
                Average_Price
                format=dollar8.2
        from sql.invoice;
quit;

Product Number,Average_Price
5001,$299.00
6001,$799.00
2101,$35.00
1110,$3200.00
5002,$399.00
6000,$99.00
1200,$3300.00


## Predicates
**Predicates** are used in PROC SQL to perform direct comparisons between two conditions or expressions.
* **BETWEEN**
* **IN**
* **IS NULL**
* **IS MISSING**
* **LIKE**
* **EXISTS**

In [17]:
%%SAS
*Select the products costing between $200.00 and $500.00 inclusively 
 from the PRODUCTS table;
proc sql;
    select prodname,
           prodtype,
           prodcost
        from sql.products
            where prodcost between 200 and 500;
quit;

Product Name,Product Type,Product Cost
Spreadsheet Software,Software,$299.00
Database Software,Software,$399.00
Wordprocessor Software,Software,$299.00
Graphics Software,Software,$299.00


In [18]:
%%SAS
*Select the products that were ordered between the years 1999 and 2000;
proc sql;
    select prodnum,
           invenqty,
           orddate
        from sql.inventory;
            where year(orddate) between 1999 and 2000;
quit;

Product Number,Inventory Quantity,Date Inventory Last Ordered
1110,20,09/01/2000
1700,10,08/15/2000
5001,5,08/15/2000
5002,3,08/15/2000
5003,10,08/15/2000
5004,20,09/01/2000
5001,2,09/01/2000


In [19]:
%%SAS
*Select the products that wered ordered between 1999 and 200 
 or where inventory quantities are greater than 15;
proc sql;
    select prodnum,
           invenqty,
           orddate
        from sql.inventory
            where (year(orddate) between 1999 and 2000) or
                   invenqty > 15;
quit;

Product Number,Inventory Quantity,Date Inventory Last Ordered
1110,20,09/01/2000
1700,10,08/15/2000
5001,5,08/15/2000
5002,3,08/15/2000
5003,10,08/15/2000
5004,20,09/01/2000
5001,2,09/01/2000


In [20]:
%%SAS
*Select all phone products from the PRODUCTS table;
proc sql;
    select prodname,
           prodtype,
           prodcost
        from sql.products
            where upcase(prodtype) = 'PHONE';
quit;

Product Name,Product Type,Product Cost
Analog Cell Phone,Phone,$35.00
Digital Cell Phone,Phone,$175.00
Office Phone,Phone,$130.00


In [21]:
%%SAS
*Select both phone and software products from the PRODUCTS table;
proc sql;
    select prodname,
           prodtype,
           prodcost
        from sql.products
            where upcase(prodtype) in ('PHONE', 'SOFTWARE');
quit;

Product Name,Product Type,Product Cost
Analog Cell Phone,Phone,$35.00
Digital Cell Phone,Phone,$175.00
Office Phone,Phone,$130.00
Spreadsheet Software,Software,$299.00
Database Software,Software,$399.00
Wordprocessor Software,Software,$299.00
Graphics Software,Software,$299.00


In [22]:
%%SAS
*Select the products that are out-of-stock from the INVENTORY table;
proc sql;
    select prodnum,
           invenqty,
           invencst
        from sql.inventory
            where invenqty is null /*is missing*/;
quit;

In [23]:
%%SAS
*Select the products that are currently stocked from the INVENTORY table;
proc sql;
    select prodnum,
           invenqty,
           invencst
        from sql.inventory
            where invenqty is not null /*is not missing*/;
quit;

Product Number,Inventory Quantity,Inventory Cost
1110,20,"$45,000.00"
1700,10,"$28,000.00"
5001,5,"$1,000.00"
5002,3,$900.00
5003,10,"$2,000.00"
5004,20,"$1,400.00"
5001,2,"$1,200.00"


In [24]:
%%SAS
*Select all product names that start with the upperchase character 'A';
proc sql;
    select prodname
        from sql.products;
            where prodname like 'A%';
quit;

Product Name
Dream Machine
Business Machine
Travel Laptop
Analog Cell Phone
Digital Cell Phone
Office Phone
Spreadsheet Software
Database Software
Wordprocessor Software
Graphics Software


In [25]:
%%SAS
*Select all products whose name contains the word 'Soft' in its name;
proc sql;
    select prodname,
           prodtype,
           prodcost
        from sql.products
            where prodtype like '%Soft%';
quit;

Product Name,Product Type,Product Cost
Spreadsheet Software,Software,$299.00
Database Software,Software,$399.00
Wordprocessor Software,Software,$299.00
Graphics Software,Software,$299.00


In [26]:
%%SAS
*Select all products whose name has trailing blanks;
proc sql;
    select prodname
        from sql.products;
            where prodname like '% ';
quit;

Product Name
Dream Machine
Business Machine
Travel Laptop
Analog Cell Phone
Digital Cell Phone
Office Phone
Spreadsheet Software
Database Software
Wordprocessor Software
Graphics Software


In [27]:
%%SAS
*Select all products whose product type has five letters and 
 starts with an uppercase character 'P';
proc sql;
    select prodname,
           prodtype,
           prodcost
        from sql.products;
            where upcase(prodtype) like 'P____';
quit;

Product Name,Product Type,Product Cost
Dream Machine,Workstation,"$3,200.00"
Business Machine,Workstation,"$3,300.00"
Travel Laptop,Laptop,"$3,400.00"
Analog Cell Phone,Phone,$35.00
Digital Cell Phone,Phone,$175.00
Office Phone,Phone,$130.00
Spreadsheet Software,Software,$299.00
Database Software,Software,$399.00
Wordprocessor Software,Software,$299.00
Graphics Software,Software,$299.00


In [28]:
%%SAS
*Select all products whose name has the lowercase character 'a' in the fourth position;
proc sql;
    select prodname
        from sql.products
            where prodname like '___a%';
quit;

Product Name
Dream Machine
Database Software
